In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,mapping,LineString, Point
import math
import tqdm
import sqlite3
from calculo_predial import *
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from Herramientas import *

In [2]:
# Ruta a los shapes
# Modelo IA
path_modelo_casas = '/home/hector/Documentos/Infis/Geo/Data/Shapes/prueba_aleatoria_Naucalpan.shp'
# Manzanas del municipio
path_manzanas     = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Manzana_Naucalpan.shp'
# Cruce de valores unitarios vs Manzana
path_vu_and_mz    = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Valores_Unitarios_con_Manzanas_Naucalpan.shp'
# Lista de afirmaciones en minisculas
afirmativo = np.array(['si','1',1,'yes','claro','obvio','of course','sip','sipi','neta','ok'])

## Proceso 1
Asociar un valor unitario a cada registro del modelo IA, para poder aproximar su valor catastral

In [3]:
# Cargamos la tabla de valores unitarios con Manzanas
cruce1 = gpd.read_file(path_vu_and_mz).to_crs(3857)

In [4]:
# Revisamos las columnas para llenar la sig celda
cruce1.columns

Index(['Tipo calle', 'Nombre Cal', 'Zona_Banda', 'Manzanas_B', 'COD_BandaV',
       'Val m2_Ban', 'Pseudo_cla', 'Zona_Manz', 'Manzanas_M', 'COD_Manz',
       'Tipo', 'Frente', 'Fondo', 'Area_AH', 'Val m2_Man', 'Pseudo_c_1', 'mun',
       'zona', 'manz', 'cve_cat', 'key', 'Lon', 'Lat', 'Tipo_cruce',
       'Tipo_cru_1', 'geometry'],
      dtype='object')

In [5]:
# Definimos columnas utiles
# Valores unitarios para banda de valores
col_vu_bv = 'Val m2_Ban'
# Valores unitarios para area homogenea
col_vu_ah = 'Val m2_Man'

In [6]:
# Revision de valores 0, no permitidos en ambos Val m2
cruce1[(cruce1[col_vu_ah].fillna(0)==0)&(cruce1[col_vu_bv].fillna(0)==0)].shape

(0, 26)

In [7]:
# Cargamos el GeoDataFrame del modelo de la IA
gdf_modelo_c = gpd.read_file(path_modelo_casas).to_crs(3857)
gdf_modelo_c.rename(columns={'area':'Area_Modelo'}, inplace=True)
print('Shape original -> ',gdf_modelo_c.shape)
gdf_modelo_c.drop_duplicates(subset='geometry', inplace=True)
print('Shape final -> ',gdf_modelo_c.shape)

Shape original ->  (50, 3)
Shape final ->  (50, 3)


In [8]:
# Intentamos borrar la clase carros si es que existe
existe_categoria = False
try:
    s = gdf_modelo_c.shape[0]
    gdf_modelo_c.drop(index=gdf_modelo_c[gdf_modelo_c['clase_dete']=='carros'].index, inplace=True)
    print('Se borraron: ',s-gdf_modelo_c.shape[0])
    # Columnas de categorias
    casas = ['casas', 'establecimiento','multivivienda']
    terrenos = ['area_verde','terreno_baldio',]
    existe_categoria = True
except:
    print('Revise que la columnas para borrar carros exista')
    print(gdf_modelo_c.columns)

Revise que la columnas para borrar carros exista
Index(['conf', 'Area_Model', 'geometry'], dtype='object')


In [9]:
# Cruce de modelo IA vs Valores Unitarios & mz
cruce2 = gpd.sjoin(cruce1, gdf_modelo_c, how='right', lsuffix='_Cruce1', rsuffix='_ModeloIA')
#cruce2['Tipo_cruce1'] = cruce2['Tipo_cruce1'].astype(str)
#cruce2['Tipo_cruce2'] = cruce2['Tipo_cruce2'].astype(str)
print('Shape original = ', cruce2.shape)
cruce2.reset_index(drop=True,inplace=True)
cruce2.dropna(subset=['cve_cat','geometry'], inplace=True)
print('Shape final    = ', cruce2.shape) 

Shape original =  (54, 29)
Shape final    =  (49, 29)


In [10]:
# Volvemos a revisar que los val por m2 esten llenos
cruce2[(cruce2[col_vu_bv].fillna(0)==0)&(cruce2[col_vu_ah].fillna(0)==0)]

,index__Cruce1,Tipo calle,Nombre Cal,Zona_Banda,Manzanas_B,COD_BandaV,Val m2_Ban,Pseudo_cla,Zona_Manz,Manzanas_M,...,manz,cve_cat,key,Lon,Lat,Tipo_cruce,Tipo_cru_1,conf,Area_Model,geometry


In [11]:
# Obtener las medidas de los lados
cruce2['Medida_lx'] = cruce2['geometry'].map(lambda x: get_lados_xy(x)[0]) 
cruce2['Medida_ly'] = cruce2['geometry'].map(lambda x: get_lados_xy(x)[1]) 

In [12]:
# Borrar cols inecesarias
cols_drop = ['index__Cruce1','key']
print('Shape original: ', cruce2.shape)
cruce2.drop(columns=cols_drop,inplace=True)
print('Shape final: ', cruce2.shape)

Shape original:  (49, 31)
Shape final:  (49, 29)


## Proceso 2
Hacer una identificacion geodesica de cada registro en el modelo IA para poder obtener factores necesarios para la aproximacion del valor catastral

In [13]:
# Hacemos un cambio de variable para mejor visualizacion
gdf = cruce2
gdf.reset_index(drop=False, inplace=True)
gdf.rename(columns={'index':'Indice_gdf'}, inplace=True)
gdf = gdf.to_crs(3857)
gdf['Esquinero|Intermedio'] = 0
gdf.shape

(49, 31)

In [14]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del cruce2, cruce1, gdf_modelo_c
        print('Matamos las variables cruce2, cruce1, gdf_modelo_c, ya no hay vuelta atras')

Matamos las variables cruce2, cruce1, gdf_modelo_c, ya no hay vuelta atras


In [15]:
# Cargamos el shape de manzanas para analisis
shape = gpd.read_file(path_manzanas)
shape=shape.to_crs("3857")
shape["centroid"]=shape.centroid
shape.shape

(6315, 6)

In [16]:
# Volvemos a revisar que los val por m2 esten llenos
gdf[(gdf[col_vu_bv].fillna(0)==0)&(gdf[col_vu_ah].fillna(0)==0)]

,Indice_gdf,Tipo calle,Nombre Cal,Zona_Banda,Manzanas_B,COD_BandaV,Val m2_Ban,Pseudo_cla,Zona_Manz,Manzanas_M,...,Lon,Lat,Tipo_cruce,Tipo_cru_1,conf,Area_Model,geometry,Medida_lx,Medida_ly,Esquinero|Intermedio


In [17]:
# Buscamos por distancias los mas cercanos al perimetro de la manzana

nuevos = get_polygons_nearest_perimeter(falta_fp=gdf,
                                        shape=shape,
                                        col_cve_cat='cve_cat',
                                        distancia_max=4,
                                        col_id='Indice_gdf')

100%|██████████| 49/49 [00:00<00:00, 134.93it/s]


 Faltan  49  Claves y se revisaron  52  Claves
Se encontraron:  21 
 Shape final -> (21, 2)


In [18]:
# etiquetamos los valores que son corner_polygons con el otro algoritmo
gdf.loc[gdf['Indice_gdf'].isin(np.unique(nuevos['Indice_gdf'].values)), 'Esquinero|Intermedio'] = 1

In [19]:
# Revision de conteos nuevos en el gdf
gdf['Esquinero|Intermedio'].value_counts()

Esquinero|Intermedio
0    28
1    21
Name: count, dtype: int64

In [20]:
# Borrar duplicados
print('Shape original ', gdf.shape)
gdf.drop_duplicates(subset=['geometry','cve_cat'],inplace=True)
print('Shape fianl ', gdf.shape)

Shape original  (49, 31)
Shape fianl  (49, 31)


In [21]:
# Volvemos a revisar que los val por m2 esten llenos
gdf[(gdf[col_vu_bv].fillna(0)==0)&(gdf[col_vu_ah].fillna(0)==0)]

,Indice_gdf,Tipo calle,Nombre Cal,Zona_Banda,Manzanas_B,COD_BandaV,Val m2_Ban,Pseudo_cla,Zona_Manz,Manzanas_M,...,Lon,Lat,Tipo_cruce,Tipo_cru_1,conf,Area_Model,geometry,Medida_lx,Medida_ly,Esquinero|Intermedio


## Proceso 3
Ahora ya tenemos la info necesaria para el calculo de los factores lo que nos llevara al valor catastral

Todo este procedimiento a implementar es un seguimiento detallado del [manual catastral](!https://igecem.edomex.gob.mx/sites/igecem.edomex.gob.mx/files/files/ArchivosPDF/Servicios-catastrales/Manual%20Catastral%20del%20Estado%20de%20Mexico.pdf) que el IGECEM publica para el calculo del valor catastral. 

In [22]:
datos = gdf
datos.fillna(0,inplace=True)

In [23]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del gdf
        print('Matamos las variables cgdf   , ya no hay vuelta atras')

Matamos las variables cgdf   , ya no hay vuelta atras


In [24]:
# borrar duplicados again
print('Shape original -> ',datos.shape)
datos.drop_duplicates(subset=['geometry','cve_cat'],inplace=True)
print('Shape nuevo    -> ',datos.shape)

Shape original ->  (49, 31)
Shape nuevo    ->  (49, 31)


In [25]:
vu_const = pd.read_excel('./Data/Valor de construccion unitario habitacional naucalpan.xlsx')
vu_const.shape

(27, 4)

In [26]:
# Obtenemos las columnas de altura, nivel, y valores para borrarlas, ya que se hara de nuevo la clasificacion
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_otros = np.array([])
col_categoria = None
datos.fillna(0, inplace=True)


for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    elif col in list(vu_const['Categoría']):
        cols_otros = np.append(cols_otros, col)
    elif col.lower().strip().startswith('clase_dete'):
        col_categoria = col
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_otros), col_categoria

(0, 0, 2, 0, None)

In [27]:
# Diccionarios utiles para clasificacion
dict_intervalos = {
    'Medio'    :(60,250),'Popular'  :(40,80),
    'Interes'  :(0,70),'Campestre':(50, 400),
    'Rural'    :(50, 400),'Comercial':(400,10000000000000000)
}
dict_niveles = {
    'NivelesM_HMEA' :18,'NivelesM_HMEM' :10,
    'NivelesM_HMEB' :4,'NivelesP_HPOM' :8,
    'NivelesP_HPOB' :6,'NivelesIS_HISM':8,
    'NivelesIS_HISB':6,'NivelesC_HCAM' :2,
    'NivelesR_HRUR' :3,'NivelesCom'    :14
}
dict_alturas = {
    'AlturaM_HMEA' :75.6,'AlturaM_HMEM' :42,
    'AlturaM_HMEB' :16.8,'AlturaP_HPOM' :33.6,
    'AlturaP_HPOB' :25.2,'AlturaIS_HISM':33.6,
    'AlturaIS_HISB':25.2,'AlturaC_HCAM' :8.4,
    'AlturaR_HRUR' :12.6,'AlturaCom'    :35
}
dict_construccion = {}
for i in range(len(vu_const)):
  dict_construccion[str(vu_const['Categoría'][i])]=(vu_const['min'][i],vu_const['Max'][i])

dict_vu_construccion = {}
for i in range(len(vu_const)):
    dict_vu_construccion['VU_'+str(vu_const['Categoría'][i])] = vu_const['Valor m2'][i]

In [28]:
dict_vu_construccion

{'VU_Precaria baja': 1217,
 'VU_Precaria media': 1794,
 'VU_Precaria alta': 2443,
 'VU_Economica baja': 3628,
 'VU_Economica media': 4395,
 'VU_Economica alta': 5044,
 'VU_Interes social baja': 5093,
 'VU_Interes social media': 6316,
 'VU_Interes social alta': 6661,
 'VU_Regular baja': 7308,
 'VU_Regular media media': 7969,
 'VU_Regular alta alta': 9517,
 'VU_Buena  baja': 10571,
 'VU_Buena  media': 11379,
 'VU_Buena  alta': 13121,
 'VU_Muy buena baja': 14520,
 'VU_Muy buena media': 16082,
 'VU_Muy buena alta': 19261,
 'VU_Lujo baja': 20519,
 'VU_Lujo media': 24163,
 'VU_Lujo alta': 27337,
 'VU_Comercial economica': 2596,
 'VU_Comercial regular': 6221,
 'VU_Comercial buena': 10225,
 'VU_Comercial muy buena': 15591,
 'VU_Comercial lujo': 22138,
 'VU_Comercial departamental': 5591}

In [29]:
# Renombramos en caso de ser necesario
renombres = {'Area_H':'Area_AH'}
datos.rename(columns=renombres,inplace=True)

In [30]:
# Columnas a utilizar, pueden variar, por eso se definen aqui
col_area_modelo = 'Area_Model'
col_area_h = 'Area_AH'

In [31]:
# Categorizar superficies por valor de construccion
import warnings
warnings.simplefilter('ignore')
for key, val in dict_construccion.items():
    print(key, val)
    if existe_categoria:
        datos.loc[datos[col_categoria].isin(casas),key] = datos[datos[col_categoria].isin(casas)][col_area_modelo].map(lambda x: Categorizar_por_intervalos(x, val[0], val[1]) )  
    else:
        datos[key] = datos[col_area_modelo].map(lambda x: Categorizar_por_intervalos(x, val[0], val[1]) )

Precaria baja (0, 40)
Precaria media (0, 40)
Precaria alta (0, 40)
Economica baja (40, 50)
Economica media (50, 60)
Economica alta (60, 80)
Interes social baja (50, 60)
Interes social media (59, 61)
Interes social alta (60, 70)
Regular baja (60, 90)
Regular media media (90, 120)
Regular alta alta (90, 120)
Buena  baja (60, 90)
Buena  media (90, 120)
Buena  alta (95, 140)
Muy buena baja (95, 140)
Muy buena media (100, 180)
Muy buena alta (180, 200)
Lujo baja (180, 200)
Lujo media (180, 200)
Lujo alta (200, 250)
Comercial economica (250, 1000000000000)
Comercial regular (250, 1000000000000)
Comercial buena (250, 1000000000000)
Comercial muy buena (250, 1000000000000)
Comercial lujo (250, 1000000000000)
Comercial departamental (250, 1000000000000)


In [32]:
datos['Comercial buena'].value_counts()

Comercial buena
0    41
1     8
Name: count, dtype: int64

In [33]:
# Categorizar superficies
for key, val in dict_intervalos.items():
    print(key, val)
    datos[key] = datos[col_area_modelo].map(lambda x: Categorizar_por_intervalos(x, val[0], val[1]) )

Medio (60, 250)
Popular (40, 80)
Interes (0, 70)
Campestre (50, 400)
Rural (50, 400)
Comercial (400, 10000000000000000)


In [34]:
# Poner niveles maximos
for key,val in dict_niveles.items():
    print(key,val)
    datos[key] = val
    if key.find('AlturaM')>=0:
        datos[key] = datos[key]*datos['Medio']
    elif key.find('AlturaP') >= 0:
        datos[key] = datos[key]*datos['Popular']
    elif key.find('AlturaIS') >= 0:
        datos[key] = datos[key]*datos['Interes']
    elif key.find('AlturaC') >= 0:
        datos[key] = datos[key]*datos['Campestre']
    elif key.find('AlturaR') >= 0:
        datos[key] = datos[key]*datos['Rural']
    elif key.find('AlturaCom') >= 0:
        datos[key] = datos[key]*datos['Comercial']
datos['Nivel_minimo'] = 1

NivelesM_HMEA 18
NivelesM_HMEM 10
NivelesM_HMEB 4
NivelesP_HPOM 8
NivelesP_HPOB 6
NivelesIS_HISM 8
NivelesIS_HISB 6
NivelesC_HCAM 2
NivelesR_HRUR 3
NivelesCom 14


In [35]:
# Poner alturas maximas
for key,val in dict_alturas.items():
    print(key,val)
    datos[key] = val
    if key.find('NivelesM')>=0:
        datos[key] = datos[key]*datos['Medio']
    elif key.find('NivelesP') >= 0:
        datos[key] = datos[key]*datos['Popular']
    elif key.find('NivelesIS') >= 0:
        datos[key] = datos[key]*datos['Interes']
    elif key.find('NivelesC') >= 0:
        datos[key] = datos[key]*datos['Campestre']
    elif key.find('NivelesR') >= 0:
        datos[key] = datos[key]*datos['Rural']
    elif key.find('NivelesCom') >= 0:
        datos[key] = datos[key]*datos['Comercial']
datos['Altura_min'] = 2.5

AlturaM_HMEA 75.6
AlturaM_HMEM 42
AlturaM_HMEB 16.8
AlturaP_HPOM 33.6
AlturaP_HPOB 25.2
AlturaIS_HISM 33.6
AlturaIS_HISB 25.2
AlturaC_HCAM 8.4
AlturaR_HRUR 12.6
AlturaCom 35


In [36]:
# Poner valores unitarios de construccion

cols_vu_cons = np.array([])
for key, val in dict_vu_construccion.items():
    print(key,val)
    nombre_cat = str(key).split('_')[-1]
    datos.loc[(datos[nombre_cat]!=0), key] = val
    cols_vu_cons = np.append(cols_vu_cons,key)

VU_Precaria baja 1217
VU_Precaria media 1794
VU_Precaria alta 2443
VU_Economica baja 3628
VU_Economica media 4395
VU_Economica alta 5044
VU_Interes social baja 5093
VU_Interes social media 6316
VU_Interes social alta 6661
VU_Regular baja 7308
VU_Regular media media 7969
VU_Regular alta alta 9517
VU_Buena  baja 10571
VU_Buena  media 11379
VU_Buena  alta 13121
VU_Muy buena baja 14520
VU_Muy buena media 16082
VU_Muy buena alta 19261
VU_Lujo baja 20519
VU_Lujo media 24163
VU_Lujo alta 27337
VU_Comercial economica 2596
VU_Comercial regular 6221
VU_Comercial buena 10225
VU_Comercial muy buena 15591
VU_Comercial lujo 22138
VU_Comercial departamental 5591


In [37]:
# Obtenemos las columnas de altura, nivel, y valores
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_const = np.array([])
datos.fillna(0, inplace=True)

for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    elif col in list(vu_const['Categoría']):
        cols_const = np.append(cols_const,col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_const)

(11, 11, 2, 27)

In [38]:
# Campos faltantes
datos['Frente_base'] = datos['Frente']
datos['Fondo_base']  = datos['Fondo']
datos['Area_base']   = datos[col_area_h]

if existe_categoria:
    datos.loc[datos[col_categoria].isin(casas),'grado_conservacion'] = datos.loc[datos[col_categoria].isin(casas)][col_area_h].map(lambda x:get_grado_conservacion())
else:
    datos['grado_conservacion'] = datos[col_area_h].map(lambda x:get_grado_conservacion())
datos['Area_inscrita']     = datos[col_area_modelo].map(lambda x: float(x)*random.uniform(0.65,1))
datos['Area_construccion'] = datos[col_area_modelo]*.9

In [39]:
# Columnas a utilizar, pueden variar, por eso se definen aqui
col_esquinero = 'Esquinero|Intermedio'

In [40]:
# Factores catastral del terreno

datos.fillna(0, inplace=True)
cols_factor_top = np.array([])

datos['Factor_posicion']      = datos[col_esquinero].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Interior' if x==0.5 else 'otro')
datos['factor_frente']        = datos.apply(lambda x: factor_frente(x['Medida_lx'], x['Terreno_Posicion']),axis=1)
datos['factor_fondo']         = datos.apply(lambda x: factor_fondo(x['Medida_ly'], x['Fondo_base'], x['Terreno_Posicion']), axis=1)

for col in cols_altura:
    nombre_col = 'Factor_Topografia_'+col
    cols_factor_top = np.append(cols_factor_top, nombre_col)
    datos[nombre_col] = datos.apply(lambda x: factor_topografia(x[col], x['Medida_ly'], x['Terreno_Posicion']), axis=1)


datos['factor_irregularidad'] = datos.apply(lambda x: factor_irregularidad(x[col_area_modelo], x['Area_inscrita'], x['Terreno_Posicion']), axis=1)
datos['factor_area']          = datos.apply(lambda x: factor_area(x[col_area_modelo],x['Area_base'],x['Terreno_Posicion']), axis=1)
datos['factor_restriccion']   = datos.apply(lambda x: factor_restriccion(x[col_area_modelo],x[col_area_modelo]*0.8,x['Terreno_Posicion']), axis=1)

In [41]:
# Revison de valores 0 en los factores de terreno y correccion

cols_ft = {'factor_frente':0.5,'factor_fondo':0.6,'factor_irregularidad':0.5,'factor_area':0.7,'factor_restriccion':0.5}

for col, val in cols_ft.items():
    s = datos[datos[col].fillna(0)==0].shape
    
    if s[0] != 0:
        datos.loc[datos[col].fillna(0)==0, col] = val
        print(col)
    print(s)
    
for col in cols_factor_top:
    s = datos[datos[col].fillna(0)==0].shape
    if s[0] != 0:
        print(col)
    print(s)

(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)
(0, 137)


In [42]:
# Factores catastral de la construcción

cols_factor_nvl = np.array([])

if existe_categoria:
    datos.loc[datos[col_categoria].isin(casas),'factor_grado_conservacion'] = datos[datos[col_categoria].isin(casas)]['grado_conservacion'].map(lambda x: factor_grado_conservacion(x))
    datos.loc[datos[col_categoria].isin(casas),'factor_edad']               = 0.6
else:
    datos['factor_grado_conservacion'] = datos['grado_conservacion'].map(lambda x: factor_grado_conservacion(x))
    datos['factor_edad']               = 0.6


for col in cols_nivel:
    nombre = 'factor_numero_niveles_'+col
    cols_factor_nvl = np.append(cols_factor_nvl, nombre)
    if existe_categoria:
        datos.loc[datos[col_categoria].isin(casas),nombre] = datos[datos[col_categoria].isin(casas)].apply(lambda x: factor_numero_niveles(x[col],x['grado_conservacion']), axis=1)
    else:
        datos[nombre] = datos.apply(lambda x: factor_numero_niveles(x[col],x['grado_conservacion']), axis=1)

In [43]:
# Revision de valores 0 en los factores de construccion
if existe_categoria:
    print(datos[(datos[col_categoria].isin(casas))&(datos['factor_grado_conservacion'].fillna(0)==0.0)].shape)
    print(datos[(datos[col_categoria].isin(casas))&(datos['factor_edad'].fillna(0)==0.0)].shape)
    
    for col in cols_factor_nvl:
        s = datos[(datos[col_categoria].isin(casas))&(datos[col].fillna(0)==0.0)].shape
        if s[0] != 0:
            print(col)
        print(s)
else:
    print(datos[(datos['factor_grado_conservacion'].fillna(0)==0.0)].shape)
    print(datos[(datos['factor_edad'].fillna(0)==0.0)].shape)
    
    for col in cols_factor_nvl:
        s = datos[(datos[col].fillna(0)==0.0)].shape
        if s[0] != 0:
            print(col)
        print(s)



(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)
(0, 150)


In [44]:
# Conversion a flotantes de los valores que necesitamos

cols_using = [col_area_modelo,'factor_frente','factor_fondo', 'factor_irregularidad','factor_area',
              'Factor_posicion','factor_restriccion']
datos.fillna(0,inplace=True)
for col in cols_using:
    datos[col] = datos[col].astype(float)

![alt text](Data/Valor_cat_terreno.png) 

Formula para obtener el valor catastral del terreno

In [45]:
# Valor catastral del terreno

cols_vc_terr = np.array([])

for col_vu in cols_valor_uni:
    name = col_vu.split('_')[-1]
    
    for col_ft in cols_factor_top:
        name_col = 'Valor_catastral_terreno_'+name+'_'+col_ft.split('_')[-1]
        datos[name_col] = datos[col_area_modelo].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_frente'].astype(float).fillna(0)*datos['factor_fondo'].fillna(0)*datos['factor_irregularidad'].fillna(0)*datos['factor_area'].fillna(0)*datos[col_ft].fillna(0)*datos['Factor_posicion'].fillna(0)*datos['factor_restriccion'].fillna(0)
        cols_vc_terr = np.append(cols_vc_terr,name_col)

![alt text](Data/Valor_cat_const.png) <br>
 Formula para obtener el valor catastral de la construccion

In [46]:
# Valor catastral de la construccion
cols_vc_const = np.array([])
for col_vu in cols_vu_cons:
    i = 0
    name = col_vu.split('_')[-1]
    for col_nv in cols_factor_nvl:
        name_col = 'Valor_catastral_construccion_'+name+'_'+str(i)
        cols_vc_const = np.append(cols_vc_const,name_col)
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_edad'].astype(float).fillna(0)*datos['factor_grado_conservacion'].astype(float).fillna(0)*datos[col_nv].astype(float).fillna(0)
        i += 1

In [47]:
# Revicion del total de columnas creadas
print(f'Cols construccion -> {len(cols_vc_const)} \nCols terreno      -> {len(cols_vc_terr)} \nShape Final       -> {datos.shape}')

Cols construccion -> 297 
Cols terreno      -> 22 
Shape Final       -> (49, 469)


In [ ]:
jhkjhl1

#### Checkpoint opcional
 <br>Hasta este punto hemos aproximado tantos valores catastrales como el numero de columnas creadas arriba.
El siguiente proceso consume muchos recurso asi que se recomienda llevarlo a google colab

In [ ]:
nombre_file = 'New_casas_Naucalpan_proces2_v1'

In [ ]:
# Crear en CSV para no truncar el nombre de columnas
pd.DataFrame(datos).to_csv('Data/Completados/Modelo_Final/'+nombre_file+'.csv',
                           index=False,
                           encoding='utf8')

Exception ignored in: <function ZipFile.__del__ at 0x7f2d63a3bdc0>
Traceback (most recent call last):
  File "/home/hector/anaconda3/envs/Infis/lib/python3.9/zipfile.py", line 1814, in __del__
    def __del__(self):
KeyboardInterrupt: 


OSError: Cannot save file into a non-existent directory: 'Data/Completados/Modelo_Final'

In [ ]:
# Crear el geopandas solo con las columnas utiles para reponerselas al csv
datos[['geometry', 'cve_cat','Indice_gdf']].to_file('Data/Completados/Modelo_Final/Recuperacion_geometry/'+nombre_file+'.shp',
                                         index=False)

#### Checkpoint opcional

In [49]:
# Camnibio de variable otra vez
df = datos
df.reset_index(drop=False, inplace=True)
df.rename(columns={'index':'Indice_Modelo'}, inplace=True)
df.shape

(49, 470)

In [50]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del datos
        print('Matamos las variables datos   , ya no hay vuelta atras')

Matamos las variables datos   , ya no hay vuelta atras


In [51]:
# Obtenemos las columnas de interes
cols_terreno = np.array([])
cols_const = np.array([])
for col in df.columns:
    if col.lower().find('valor_catastral_construccion')>=0:
        cols_const = np.append(cols_const, col)
    elif col.lower().find('valor_catastral_terreno')>=0:
        cols_terreno = np.append(cols_terreno, col)
len(cols_terreno), len(cols_const)

(22, 297)

In [52]:
# Volvemos a revisar que los val por m2 esten llenos
df[(df['Val m2_Ban'].fillna(0)==0)&(df['Val m2_Man'].fillna(0)==0)]

,Indice_Modelo,Indice_gdf,Tipo calle,Nombre Cal,Zona_Banda,Manzanas_B,COD_BandaV,Val m2_Ban,Pseudo_cla,Zona_Manz,...,Valor_catastral_construccion_Comercial departamental_1,Valor_catastral_construccion_Comercial departamental_2,Valor_catastral_construccion_Comercial departamental_3,Valor_catastral_construccion_Comercial departamental_4,Valor_catastral_construccion_Comercial departamental_5,Valor_catastral_construccion_Comercial departamental_6,Valor_catastral_construccion_Comercial departamental_7,Valor_catastral_construccion_Comercial departamental_8,Valor_catastral_construccion_Comercial departamental_9,Valor_catastral_construccion_Comercial departamental_10


In [53]:
# Obtenemos los estadisticos de interes, esta parte es la mas pesada
df['Estadisticos_terreno'] = df.apply(lambda x: statistics_values(x[cols_terreno]), axis=1)
df[df['Estadisticos_terreno']=='Revisar'].shape

(0, 471)

In [54]:
# Obtenemos los estadisticos de interes, esta parte es la mas pesada
df['Estadisticos_construccion'] = df.apply(lambda x: statistics_values(x[cols_const]), axis=1)
df[df['Estadisticos_construccion']=='Revisar'].shape

(0, 472)

In [ ]:
df['Len']

In [55]:
# Damos formatoa las cols Minimo, Maximo, Media, desviacion_estandar, largo, media_percentil, desviacion_std_percentiles, moda_percentiles
df = pd.concat([df,
                df['Estadisticos_terreno'].str.split('|', expand=True).rename(columns={0:   'Min_terreno',
                                                                                       1:   'Max_terreno',
                                                                                       2: 'Media_terreno',
                                                                                       3:   'STD_terreno',
                                                                                       4:   'Len_terreno',
                                                                                       5:'MediaP_terreno',
                                                                                       6:  'STDP_terreno',
                                                                                       7: 'ModaP_terreno'})], axis=1)

df = pd.concat([df,
                df['Estadisticos_construccion'].str.split('|', expand=True).rename(columns={0:   'Min_construccion',
                                                                                            1:   'Max_construccion',
                                                                                            2: 'Media_construccion',
                                                                                            3:   'STD_construccion',
                                                                                            4:   'Len_construccion',
                                                                                            5:'MediaP_construccion',
                                                                                            6:  'STDP_construccion',
                                                                                            7: 'ModaP_construccion'})], axis=1)

Calculo del intervalo de confianza suponiendo distribucion normal en los valores catastrales

In [56]:
# Valor z con nivel de significancia al 95%
z = stats.norm.ppf(.95)
z

1.6448536269514722

![alt text](./Data/Valor_cat_final.png)

Formula para obtener el valor catastral final 

![alt text](./Data/Intervalo_confianza_norm.png) Formula para intervalo de confianza

In [58]:
cols_estadisticos = ['Media_terreno','STD_terreno','Len_terreno','Media_construccion','STD_construccion','Len_construccion']

for col in cols_estadisticos:
    try:
        df[col] = df[col].astype(float)
    except:
        print(col)

In [ ]:
df.head(2)

,Indice_Modelo,Indice_gdf,Tipo calle,Nombre Cal,Zona_Banda,Manzanas_B,COD_BandaV,Val m2_Ban,Pseudo_cla,Zona_Manz,...,STDP_terreno,ModaP_terreno,Min_construccion,Max_construccion,Media_construccion,STD_construccion,Len_construccion,MediaP_construccion,STDP_construccion,ModaP_construccion
0,0,0,0,0,0,0,0,0.0,0,07,...,5.820766091346741e-11,465976.1580841067,170276.70705731487,1498541.0186985945,689054.104153,441694.692389,66.0,744656.6665075627,454791.17833363445,172320.02754200264
1,1,1,0,0,0,0,0,0.0,0,08,...,0.0,134644.8392220021,163759.8692919493,354183.3776348775,235608.434654,60717.936326,55.0,241839.56202270556,57904.01543307831,165069.94824628488


In [59]:
df['Media_terreno']#- (z*df['STD_terreno']/df['Len_terreno'])

0     465976.158084
1     134644.839222
2     679530.366824
3     124536.948774
4     289679.039480
5      96254.002649
6      30780.773762
7     194525.980895
8     144265.444714
9      25024.629066
10     39513.403833
11      2139.366910
12     29922.854622
13     55346.845854
14     43201.085980
15     53350.972998
16     21112.237019
17     67394.626195
18     83349.918552
19     52287.250551
20     42675.739437
21    188837.494439
22     26198.828720
23     35100.882362
24     17225.003930
25     85861.854612
26     92272.265860
27    193720.425284
28    162794.243769
29     59990.598940
30    350315.068823
31    546308.548861
32     40206.465245
33    163520.656033
34    247971.837757
35    212329.456602
36     74757.919658
37     26901.015344
38       916.386672
39     33056.103689
40     65991.861914
41    414090.409348
42     94062.000883
43    184887.374958
44     10154.921940
45    318143.997447
46    426157.795622
47    549762.015628
48     62726.425290
Name: Media_terreno,

In [60]:
# Limite inferior (LI) y superior (LS) de cada valor catastral
df['LI_95%_vc_terreno'] = df['Media_terreno']- (z*df['STD_terreno']/df['Len_terreno'])
df['LS_95%_vc_terreno'] = df['Media_terreno']+ (z*df['STD_terreno']/df['Len_terreno']) 

df['LI_95%_vc_construccion'] = df['Media_construccion']- (z*df['STD_construccion']/df['Len_construccion'])
df['LS_95%_vc_construccion'] = df['Media_construccion']+ (z*df['STD_construccion']/df['Len_construccion']) 

df['LI_95%_catastral_final'] = df['LI_95%_vc_terreno'] + df['LI_95%_vc_construccion'] 
df['LS_95%_catastral_final'] = df['LS_95%_vc_terreno'] + df['LS_95%_vc_construccion'] 

In [ ]:
shape.head(10)

In [63]:
df.head(10)

,Indice_Modelo,Indice_gdf,Tipo calle,Nombre Cal,Zona_Banda,Manzanas_B,COD_BandaV,Val m2_Ban,Pseudo_cla,Zona_Manz,...,Len_construccion,MediaP_construccion,STDP_construccion,ModaP_construccion,LI_95%_vc_terreno,LS_95%_vc_terreno,LI_95%_vc_construccion,LS_95%_vc_construccion,LI_95%_catastral_final,LS_95%_catastral_final
0,0,0,0,0,0,0,0,0.0,0,07,...,66.0,1675477.4996420161,1023280.1512506775,387720.0619695059,465976.158084,465976.158084,1.525604e+06,1.575140e+06,1.991580e+06,2.041116e+06
1,1,1,0,0,0,0,0,0.0,0,08,...,55.0,107484.24978786916,25735.117970257023,73364.42144279329,134644.839222,134644.839222,1.039078e+05,1.055219e+05,2.385527e+05,2.401667e+05
2,2,3,Boulevar,Avila Camacho Manuel,01,547,321,4312.0,09801547,01,...,66.0,1373937.788093191,839117.9636309128,317941.15077973634,656320.404470,702740.329178,1.251037e+06,1.291658e+06,1.907358e+06,1.994398e+06
3,3,4,0,0,0,0,0,0.0,0,04,...,33.0,1803777.4793409936,180163.76899519135,1594901.3212083536,124536.948774,124536.948774,1.768074e+06,1.785432e+06,1.892611e+06,1.909969e+06
4,4,5,Boulevar,Avila Camacho Manuel,01,577,325,5551.0,09801577,01,...,33.0,884830.875986011,74017.94496313884,781806.0488409734,283969.921685,295388.157276,8.705806e+05,8.778607e+05,1.154551e+06,1.173249e+06
5,5,6,0,0,0,0,0,0.0,0,06,...,33.0,28928.572047665035,8476.578982215138,38543.475562877116,93570.793378,98937.211920,2.744795e+04,2.827255e+04,1.210187e+05,1.272098e+05
6,6,7,0,0,0,0,0,0.0,0,03,...,22.0,109680.24192295731,8143.103973788513,100408.93606247363,30780.773762,30780.773762,1.087198e+05,1.099326e+05,1.395006e+05,1.407134e+05
7,7,8,0,0,0,0,0,0.0,0,05,...,66.0,873903.6988998244,533727.435539901,202228.91466176958,194525.980895,194525.980895,7.957319e+05,8.215690e+05,9.902579e+05,1.016095e+06
8,8,9,0,0,0,0,0,0.0,0,07,...,11.0,1647335.9923531306,15442.30890201775,1641149.1206680723,144265.444714,144265.444714,1.643238e+06,1.647941e+06,1.787503e+06,1.792206e+06
9,9,10,0,0,0,0,0,0.0,0,09,...,11.0,84577.597716034,792.8397098007523,84259.95107507636,25024.629066,25024.629066,8.436720e+04,8.460866e+04,1.093918e+05,1.096333e+05


In [ ]:
# Solo cols de interes
df[np.append(cols_estadisticos,[])]

#### En caso de haber hecho checkpoint

In [ ]:
# En caso de haber hecho checkpoint
path_gdf =  'Data/Completados/Modelo_Final/Recuperacion_geometry/'+nombre_file+'.shp'
gdf = gpd.read_file(path_gdf)
gdf.rename(columns={'Indice_gdf':'Indice_gdf1'},inplace=True)

final = gpd.GeoDataFrame(pd.concat([gdf[['geometry', 'cve_cat','Indice_gdf1']] , df[cols_new]], axis=1), geometry='geometry')
final.tail(2)

gdf.shape, final.shape

In [ ]:
# Revision de empatamiento 
final[final['Indice_gdf']!=final['Indice_gdf1']]

In [ ]:
# Exportar el GeoDataFrame
final.to_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/Completados/Modelo_Final/Solo_VC_Final/NewCasas_Valor_Catastral_Naucalpan_Final_v1.shp',
                                                  index=False)

#### En caso de no haber hecho checkpoint

In [ ]:
# Calculo del predial
calculo_predial(83783)

197.731842

In [ ]:
# Ya terminamos solo exportamos el geodataframe
df.to_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/Completados/Modelo_Final/Solo_VC_Final/NewCasas_Valor_Catastral_Naucalpan_Final_v1.shp',
                                                  index=False)